In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
!nvidia-smi

Mon Jun 15 10:12:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 405Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 89.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 878Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 68.4Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 476Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 165Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [4]:
!ls /content/models

124M


In [5]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file_name = "Hebrew_Bible-k_encoded.npz"
!cp "/content/drive/My Drive/colab_data/gpt2/$file_name" "/content/$file_name"

run_name = "Hebrew_Bible-k"


In [0]:
#######   Do this only on the very first run
!mkdir /content/chekpoint
!cp -r "/content/drive/My Drive/colab_data/gpt2/$run_name" "/content/checkpoint/"
###################################


#######   On the folllowing runs do this instead
#gpt2.copy_checkpoint_from_gdrive(run_name=run_name)
###################################

!rm /content/models/124M/vocab.bpe
!cp "/content/checkpoint/$run_name/vocab.bpe" /content/models/124M/
!rm /content/models/124M/encoder.json
!cp "/content/checkpoint/$run_name/encoder.json" /content/models/124M/

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=5000,
              restore_from='latest',
              run_name=run_name,
              print_every=50,
              sample_every=500,
              save_every=1000,
              overwrite=True
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/Hebrew_Bible-k/model-5000
INFO:tensorflow:Restoring parameters from checkpoint/Hebrew_Bible-k/model-5000


100%|██████████| 1/1 [00:00<00:00, 20.87it/s]

Loading dataset...
dataset has 1173016 tokens
Training...
Saving checkpoint/Hebrew_Bible-k/model-5000


Saving checkpoint/Hebrew_Bible-k/model-5000
======== SAMPLE 1 ========
;
כח בימים, ההם לאלוהים:  אם-יתכפרכות את-כל-המלאכה, אשר-אני נותן הלויים--כי בידך, ועצובת ממנו ועל-כל-יהודה. 
כט ושמע, מבני-לך--למה בכפי, על-ירושלים. 
ללמיתי את-מצוות יהוה אלוהיך, לכפר לפניך; ושמע, את כל-הרעה, אשר ציוויתי, אליך. 
לא כי את-כל-דבריו יהוה בך, למען לריבי; עת, את-כל-דבריו יהוה בך, והחרש, ידעיה. 
לב כי גויים, יצאת, ונטיתי, היאספו; הדברים מלכי, אשר תעשון נכונים. 
לג לא נעדר, אשר שלח ליהוה אלוהיך, כסוי ליהוה אלוהיהם. 
לד והצתי יהוה אלוהינו, וסביב הארץ; עת ולמען האשם, ליהוה אלוהיהם. 
לה ולא-אסתיר אדוני אלוהיך, ליהוה אלוהיך:  כי אדבר, לפני יהוה אלוהיך--שאלו הוא. 
לו והייתה פלשתים, לחדש העם; והייתהים חירפת אלוהייך ודמה ברושים בכל-זרע עמון, כי עלית ליהוה. 
לז נוגף, חבקוק, שימני, ליהוה; והייתה נאנאוה ושרק, בכל-הארצות ליהוה. 
לח ויאמר יהוה אלוהיך בערב לי, חורבה ימיך, וינס ממך, לכוהן:  פן-יהיה, לאכול-לחם לגבולם. 
לט הגידו יהוה אלוהיך, לא נוכל, שופטיםים נקםות, כמוך אליי; לא-יישא ראה, בכל-וניא. 
מ ערוך כמים, מסביב; ו

In [14]:
!cat /content/checkpoint/$run_name/checkpoint

model_checkpoint_path: "model-5000"
all_model_checkpoint_paths: "model-5000"


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_name)


In [0]:
!cp -r "./checkpoint/$run_name" "/content/drive/My Drive/colab_data/gpt2/"

In [0]:
!mkdir "/content/drive/My Drive/colab_data/gpt2/$run_name-samples"
!cp -r "./samples/" "/content/drive/My Drive/colab_data/gpt2/$run_name-samples"

In [18]:
gpt2.reset_session(sess)

In [19]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Loading checkpoint checkpoint/Hebrew_Bible-k/model-5000
INFO:tensorflow:Restoring parameters from checkpoint/Hebrew_Bible-k/model-5000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [38]:
#gpt2.generate(sess, run_name=run_name, prefix="<|startoftext|>", truncate="<|endoftext|>")
gpt2.generate(sess, run_name=run_name, prefix=" ", include_prefix=False,)


א,א ויהי רעב, בארץ; וימצא בנים מתלקחת, ויישב שלמה, ארצה. 
נ,ב ויהי רעב, בארץ; וילך עשיו, שפך הגדול, ושם, הייתה בירושלים מעון. 
נ,ג ויאמר יהוה אל-משה, מות יומת האיש; והוא הבית ולהשמיד ויגידו, ושם אשת יעשו. 
נ,ד ויאמר יהוה אל-משה ואל-אהרון, מן-היום הזה--כי בכסף וישראל בי, לפניך; מי-אתה אתם היום, לפניך. 
נ,ה ויאמר עשיו--אחרי בירושלים, תרימו בהם:  שובו האיש, בגרר ירובעם; ויאמר, תפתה וגם- פצהאלה את-הבית. 
נ,ו ויהי, בבוקר, ותיפעם, בית עשיו--עד-אחוזת האיש, את-גיא מיזה; ויאמרו, אל-כל-עדת בני ישראל. 
נ,ז ויבוא יעקוב אחות יריחו, והנה יושב על-מזבח יהוה, ויאמר, לפניו, והנה:  למען, אשר, לא-המזבח את-העולה ואת-החלבים, ואת-החלבים. 
נ,ח ויאמר עשיו--אחרי-נא ידך, האנשים האלה:  אשר אמר-לו, ויקראו-לו קח-לך. 
נ,ט ויאמר, אל-העם, בניי בו על-מזבח יהוה:  כי-יבש,סח הזה אשר אתם עושים, על-ישראל. 
נ,י ויאמר, אל-משה, אני ידעתי, ביד מלך-ישראל; ונמכרוינה, על-יד עשיו, לפניו. 
נ,יא ויעמדו הדברו, כימים מעל, החייתם מהרבה, כור הארץ; אל-יהוה--אל-המחנה, יהודית. 
נ,יב ויהי, כשמוע המלך אחשוורוש, בחלום ליהוה--ככתוב, לפני יהוה;ג

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
!ln -s /content/checkpoint/$run_name /content/checkpoint/run1

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="א,א ויקח מנשה פומפרניקל את",
              #truncate="<|endoftext|>",
              include_prefix=False,
              nsamples=5,
              batch_size=5
              )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      prefix=" ",
                      #truncate="<|endoftext|>",
                      include_prefix=False,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)